In [1]:
import time
import sys
sys.path.append('/lustre/project/Stat/1155223034/atacFormer/data')

start_time = time.time()
import preprocess
print(f"Imported preprocess in {time.time() - start_time} seconds")

start_time = time.time()
import databank
print(f"Imported databank in {time.time() - start_time} seconds")

Imported preprocess in 61.20476937294006 seconds


/lustre/project/Stat/1155223034/miniconda/envs/atacformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


Imported databank in 499.0348093509674 seconds


In [2]:
from pathlib import Path
import scanpy as sc

In [3]:
import pandas as pd

In [4]:
data_dir = Path("/lustre/project/Stat/1155223034/atacFormer/data/heart")
adata_atac = sc.read_h5ad(data_dir / "HBM233.GKRM.627_ATAC.h5ad")

## The Data Structure of the sc-ATAC （Ann_data)

atac.X: cell-by-peak matrix whose entries represents is there a peak in the cell. i.e X_(i,j) = True if there is a peak j in cell i.

The property of rows are stored in the atac.obs (here the rows represent cells)

The property of columns are stored in the atac.obsm (here the columns represent peaks)



In [5]:
print(adata_atac) # cell by peak matrix, each colomn denotes a peak

AnnData object with n_obs × n_vars = 24982 × 536282
    obs: 'cluster'
    obsm: 'X_umap'


In [6]:
print(adata_atac.X[0,1])

False


In [7]:
print(adata_atac.var_names) # peak names

Index(['1:10001-15000', '1:15001-20000', '1:50001-55000', '1:55001-60000',
       '1:60001-65000', '1:65001-70000', '1:70001-75000', '1:75001-80000',
       '1:80001-85000', '1:85001-90000',
       ...
       'Un_JTFH01001780v1_decoy:1-5000', 'Un_JTFH01001781v1_decoy:1-5000',
       'Un_JTFH01001782v1_decoy:1-5000', 'Un_JTFH01001784v1_decoy:1-5000',
       'Un_JTFH01001785v1_decoy:1-5000', 'Un_JTFH01001786v1_decoy:1-5000',
       'HLA-DQA1*04:02:1-5000', 'HLA-DRB1*08:03:02:1-5000',
       'HLA-DRB1*08:03:02:5001-10000', 'HLA-DRB1*11:01:02:5001-10000'],
      dtype='object', length=536282)


In [10]:
filtered_bins_file = "/lustre/project/Stat/1155223034/atacFormer/data/bins_5k_table_23chr.txt"

In [11]:
adata_atac = preprocess.preprocessor(adata_atac, filter_bin=True,bin_file=filtered_bins_file, filter_cell_by_bins=1000)

Filtering bins ...
606199
Common bins: 530171
Filtering cells by counts ...
Binarizing data ...


In [12]:
bdata = preprocess.preprocessor(adata_atac, filter_bin=True,intersect =False, bin_file=filtered_bins_file, filter_cell_by_bins=1000)

Filtering bins ...
606199
not intersect
Current batch size: 100000


100%|██████████| 6/6 [00:42<00:00,  7.00s/it]


Filtering cells by counts ...
Binarizing data ...


In [13]:
print(bdata.var_names)

Index(['1:1-5000', '1:5001-10000', '1:10001-15000', '1:15001-20000',
       '1:20001-25000', '1:25001-30000', '1:30001-35000', '1:35001-40000',
       '1:40001-45000', '1:45001-50000',
       ...
       'X:155990001-155995000', 'X:155995001-156000000',
       'X:156000001-156005000', 'X:156005001-156010000',
       'X:156010001-156015000', 'X:156015001-156020000',
       'X:156020001-156025000', 'X:156025001-156030000',
       'X:156030001-156035000', 'X:156035001-156040000'],
      dtype='object', length=606199)


In [14]:
print(bdata)

AnnData object with n_obs × n_vars = 8221 × 606199
    obs: 'cluster', 'n_genes'
    obsm: 'X_umap'


In [15]:
print(bdata.obs['n_genes'])

0       4511
1       1760
2       1193
3       2432
4       1224
        ... 
8216    1696
8217    1992
8218    1330
8219    1161
8220    1081
Name: n_genes, Length: 8221, dtype: int64


### Summary of the preprocess.py
bin file: This can be understood as a list of bins which in our model, the DNA sequence is splitted into 5000 sequence bins.

common_bins is compring of bins which exist in both of the original atac data and the bin-file.

If not intersect, then a new bdata is created with rows representing cells in adata while the columns are bins in the bin_file. Then we fill the common bins with the data in adata corresponding to it. The non-overlapping peaks will be remained as 0.

If intersect, we pick submatrix from the original adata.X by taking the data in the common_bins.

Filtering cells by counts: filter out cells whose number of open peaks will smaller then the setting value and finally an extra information about rows in bdata.X will be added, that is obs will contain an extra key for illustrating the number of open peaks in each cell.




# Data Bank

## Data Table
创建这个class需要包含的信息： name of data table, data(DataSet)

可以调用的函数：save(path, format) 将Data Table以指定形式储存在指定的位置

## Meta Information
创建这个class需要包含on_disk_path, on_disk_format 和 main_table_key信息， 

manifests = {"on_disk_format": self.on_disk_format,"main_data": self.main_table_key, }

main_table_key表示的是哪一个数据是重要的

可以调用的函数：

（1） save(path, suffix) /path/manifest.suffix.json中保存manifests

（2） load(path) 通过path来load存在这个path上的manifests，并且将manifests里面所包含的信息放在self.on_disk_format 和 self.main_table_key中

（3） from_path(path)创建一个新的MetoInfo对象，然后将这个对象的self.on_disk_path设置为输入函数的path。然后调用load(path)函数将path下的manifest.json文件中的内容加载到自己的属性中

## Setting
创建这个class所需要的包含的初始化信息有：remove_zero_rows(T/F), max_tokenize_batch_size(integer), immediate_save(T/F)


## DataBank
创建一个DataBank class需要的初始化信息：meta_info（class MetaInf), data_tables(a dictionary; key:name of data_table, value: the corresponding data table of each key)
DataBank 可以被理解为包含着多个data table还有data table所对应的meta_information。

 (1) __post_init__(): 这个函数在创建DataBank类的时候会自动call，在只有metaInfor的时候，用sync()来自动初始化信息。 用_validate_data() 验证data tables 和 main_table_key之间的匹配性。也就是检查main_table_key在不在data_tables的key中，或者一些其他的错误。



（2） sync():用来同步data table / meta_information /both of them 会直接在data table和 meta_information储存的地方直接覆盖原本的信息。

（3） main_data():得到data_tables中main_table_key所对应的表

（4） _validate_data(): 验证data tables 和 main_table_key之间的匹配性。也就是检查main_table_key在不在data_tables的key中，或者一些其他的错误。

（5）load_anndata（adata: AnnData,data_keys: Optional[List[str]] = None,token_col: str = "gene name"） 
token_col可以用来指定一个bin的名字，如果没有指定bin就会将所有的bin的名字来形成一个tokens（a list comprising of all bins' name or the name specified by token_col)
随后得到tokens里面所有bin属于的染色体和初始碱基位置，然后对于data_keys中的每一个key调用 （6） _load_anndata_layer得到一个Dataset，然后将这些Dataset append 到data_tables which is a list comprising of data_table. Each data table belongs to DataTable class: name is the data name ("X", "raw") and the data is a dataset comprising of three columns (cell_id, non_zero_bin_chr, non_zero_bin_pos)

问题：这个函数中的chr is a list of integers while for applying function defined in (6),it should be a list of str.

（6） _load_anndata_layer(adata: AnnData, chr: List[str], pos: List[int],data_key: Optional[str] = "X",): 
data_key 可能take的值：“X” 或者是adata.layers 或 adata.obsm 中的key值。主要结构是调用7）_tokenize来得到包含"id","chr","pos" as illustrated in (7) 的dictionary，然后调用Dataset.from_dict将字典转换为dataset，每个column表示dictionary里面的一个key，一个row表示一个细胞。


（7）_tokenize(self, data: Union[np.ndarray, csr_matrix], chr: List[str], pos: List[int],) -> Dict[str, List]:
 Args:
            data (np.ndarray or spmatrix): Data to be tokenized.
            chr: list of chromosome names for each bin.
            pos: list of chromosome positions for each bin.
 Returns:
         Dict[str, List]: Tokenized data.

Tokenized data: a dictionary comprising of three values: 
"id": id of cells(the first cell will be denoted as 1), 
"chr": a list comprising of sublists which each sublist denotes the "chr" corresponding to the non-zero bin in the specific cell, 
"pos": a list comprising of sublists which each sublist denotes the initial position of the DNA sequence corresponding to the non-zero bin in the specific cell


csr_matrix：的形式
[[ 0,  5,  0,  0],
 [ 3,  0,  0,  2],
 [ 0,  0,  0,  4]]
data = [5, 3, 2, 4] #所有非零
indices = [1, 0, 3, 3] #每一个非零值在它所在的row上的index
indptr = [0, 1, 3, 4] #每行（细胞）在 data/indices 中的起始和结束位置

(8) update_datatables(self,new_tables: List[DataTable], #指定每个数据表的name use_names: List[str] = None, overwrite: bool = False, immediate_save: Optional[bool] = None,) -> None:
        """
        Update the data tables in the DataBank with new data tables.

        Args:
            new_tables (list of :class:`DataTable`): New data tables to update.
            use_names (list of :obj:`str`): Names of the new data tables to use.
                If not provided, will use the names of the new data tables.
            overwrite (:obj:`bool`): Whether to overwrite the existing data tables.
            immediate_save (:obj:`bool`): Whether to save the data immediately after
                updating. Will save to :attr:`self.meta_info.on_disk_path`. If not
                provided, will follow :attr:`self.settings.immediate_save` instead.
                Default to None.
        """
    将新的data_table以正确的方式加入原本的data_tables里面或将原本data_tables里面的data_table改写（直接覆盖）
（9）from_path(cls, path: Union[Path, str]) -> Self 将提供路径中的datatable文件转位DataTable类后全部写入一个新的DataBank类里面，新的DataBank类包含原路径中的Meta_Inf

（10）_nparray2indexed_values(data: np.ndarray （cell-by-bin matrix), chr: List[str], pos: List[int],) -> Tuple[List, List, List]:
    """
    Convert a numpy array to indexed values. Only include the non-zero values.

    Args:
        data (np.ndarray): Data matrix.
        chr: list of chromosome names for each bin.
        pos: list of chromosome positions for each bin.

    Returns:
        Tuple[List, List, List]: Row IDs, chromosome names, and chromosome positions.
    """
    当数据不是crs_matrix的时候，要怎么得到三个list：
a list comprising of id of cells(the first cell will be denoted as 1), 
a list comprising of sublists which each sublist denotes the "chr" corresponding to the non-zero bin in the specific cell, 
a list comprising of sublists which each sublist denotes the initial position of the DNA sequence corresponding to the non-zero bin in the specific cell

（11）_nparray2indexed_values_numba() 作用和（10）是完全一样的，只是速度会更快

（12）_nparray2mapped_values(）作用和(10),(11)完全一样，只是选择其中的一种来用

(13) from_anndata(cls, adata: Union[AnnData, Path, str], to: Union[Path, str], main_table_key: str = "X", token_col: Optional[str] = None, immediate_save: bool = True,) -> Self:
        """
        Create a DataBank from an AnnData object.

        Args:
            adata (AnnData): Annotated data or path to anndata file.
            to (Path or str): Data directory.
            main_table_key (str): This layer/obsm in anndata will be used as the
                main data table.
            token_col (str): Column name of the gene token.
            immediate_save (bool): Whether to save the data immediately after creation.
        Returns:
            DataBank: DataBank instance.
        """
    加载提供路径的anndata文件后，在to的路径下创建一个DataBank，将DataBank的main_table_key按照输入值设定，然后call函数（5）load_anndata（adata: AnnData,data_keys: Optional[List[str]] = None,token_col: str = "gene name"）得到 data_tables which is a list comprising of data_table. Each data table belongs to DataTable class: name is the data name ("X", "raw") and the data is a dataset comprising of three columns (cell_id, non_zero_bin_chr, non_zero_bin_pos)然后选取第一个，也就是“X”所对应的dataset。




In [16]:
sys.argv = ['build_large_scale_data.py',
            '--input-dir', '/lustre/project/Stat/1155223034/atacFormer/data/heart', 
            '--bin-file', '/lustre/project/Stat/1155223034/atacFormer/data/bins_5k_table_23chr.txt']

In [17]:
import build_large_scale_data

Found 7 files in /lustre/project/Stat/1155223034/atacFormer/data/heart

Processing HBM394.PKDH.674_ATAC.h5ad
Only considering the two last: ['.674_ATAC', '.h5ad'].
Only considering the two last: ['.674_ATAC', '.h5ad'].
originally read (2764, 533212) data from HBM394.PKDH.674_ATAC.h5ad
Filtering bins ...
606199
Common bins: 528446
Filtering cells by counts ...
Binarizing data ...
read (2763, 528446) valid data from HBM394.PKDH.674_ATAC.h5ad
DataBank - INFO - No suffix provided, saving to default.
DataBank - INFO - Saving data table X to /lustre/project/Stat/1155223034/atacFormer/data/heart/HBM394.PKDH.674.scb/datatable.parquet.


Creating parquet from Arrow format: 100%|██████████| 3/3 [00:00<00:00,  4.21ba/s]



Processing HBM656.KSMZ.578_ATAC.h5ad
Only considering the two last: ['.578_ATAC', '.h5ad'].
Only considering the two last: ['.578_ATAC', '.h5ad'].
originally read (16279, 534461) data from HBM656.KSMZ.578_ATAC.h5ad
Filtering bins ...
606199
Common bins: 531350
Filtering cells by counts ...
Binarizing data ...
read (15647, 531350) valid data from HBM656.KSMZ.578_ATAC.h5ad
DataBank - INFO - No suffix provided, saving to default.
DataBank - INFO - Saving data table X to /lustre/project/Stat/1155223034/atacFormer/data/heart/HBM656.KSMZ.578.scb/datatable.parquet.


Creating parquet from Arrow format: 100%|██████████| 16/16 [00:01<00:00, 15.25ba/s]



Processing HBM472.QPXD.658_ATAC.h5ad
Only considering the two last: ['.658_ATAC', '.h5ad'].
Only considering the two last: ['.658_ATAC', '.h5ad'].
originally read (32828, 537709) data from HBM472.QPXD.658_ATAC.h5ad
Filtering bins ...
606199
Common bins: 533441
Filtering cells by counts ...
Binarizing data ...
read (32264, 533441) valid data from HBM472.QPXD.658_ATAC.h5ad
DataBank - INFO - No suffix provided, saving to default.
DataBank - INFO - Saving data table X to /lustre/project/Stat/1155223034/atacFormer/data/heart/HBM472.QPXD.658.scb/datatable.parquet.


...

Creating parquet from Arrow format: 100%|██████████| 22/22 [00:00<00:00, 23.38ba/s]



Processing HBM477.BPMF.855_ATAC.h5ad
Only considering the two last: ['.855_ATAC', '.h5ad'].
Only considering the two last: ['.855_ATAC', '.h5ad'].
originally read (21016, 536104) data from HBM477.BPMF.855_ATAC.h5ad
Filtering bins ...
606199
Common bins: 530043
Filtering cells by counts ...
Binarizing data ...
read (18714, 530043) valid data from HBM477.BPMF.855_ATAC.h5ad
DataBank - INFO - No suffix provided, saving to default.
DataBank - INFO - Saving data table X to /lustre/project/Stat/1155223034/atacFormer/data/heart/HBM477.BPMF.855.scb/datatable.parquet.


Creating parquet from Arrow format: 100%|██████████| 19/19 [00:00<00:00, 22.67ba/s]



Processing HBM772.PDCW.864_ATAC.h5ad
Only considering the two last: ['.864_ATAC', '.h5ad'].
Only considering the two last: ['.864_ATAC', '.h5ad'].
originally read (1148, 518559) data from HBM772.PDCW.864_ATAC.h5ad
Filtering bins ...
606199
Common bins: 514897
Filtering cells by counts ...
Binarizing data ...
read (1138, 514897) valid data from HBM772.PDCW.864_ATAC.h5ad
DataBank - INFO - No suffix provided, saving to default.
DataBank - INFO - Saving data table X to /lustre/project/Stat/1155223034/atacFormer/data/heart/HBM772.PDCW.864.scb/datatable.parquet.


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 25.82ba/s]
